In [2]:
import os
from pathlib import Path

import json
import numpy as np
import pandas as pd
from Bio import SeqIO
from rdkit import Chem
from rdkit.Chem import AllChem
from IPython.display import display

cwd = os.getcwd()
if cwd.endswith('notebook/PG'):
    os.chdir('../..')
    cwd = os.getcwd()

from src.pg_modelling.ligand_utils import generate_ccd_from_smiles, sanitize_ligand_name, generate_conformation

In [3]:
data_folder = Path(os.path.expanduser('~')) / 'Documents' / 'PG_modelling'
assert data_folder.is_dir()

In [4]:
glycan_modifications = pd.read_csv(
    data_folder / 'glycan_modifications' / 'glycan_modifications.csv', 
    index_col='ligand_name',
)
print(f'Number of fragments in subset: {len(glycan_modifications):,}')
glycan_modifications

Number of fragments in subset: 9


,Name,Synonym,Formula,Monoisotopic Mass,Modifications,Degree Amidation,Degree Acetylation,Ontology,PGN Units,Glycan Units,...,Peptide,SMILES,INCHIKEY,clogP,RT,[M+H]+,[M+Na]+,[M+K]+,[M+2H]2+,[M+3H]3+
ligand_name,,,,,,,,,,,,,,,,,,,,,
G-NAM,(G)(NAM)-None,GlcN.MurNAc--None,C17H30N2O12,454.17987,Amidase,0,1,G2,1,2,...,NaN,CC(=O)NC1C(O)OC(CO)C(OC2OC(CO)C(O)C(O)C2N)C1OC...,CCRWBXATDKHQDG-UHFFFAOYSA-N,-4.7896,0,455.18715,477.16909,493.14303,228.09722,152.40057
G-DAM,(G)(DAM)-None,GlcN.MurNAcOAc--None,C19H32N2O13,496.19044,Amidase,0,2,G2,1,2,...,NaN,CC(=O)NC1C(O)OC(COC(C)=O)C(OC2OC(CO)C(O)C(O)C2...,WMTUYCBSSKTNNR-UHFFFAOYSA-N,-4.2188,0,497.19772,519.17966,535.15360,249.10250,166.40409
NAG-MUR,(NAG)(MUR)-None,GlcNAc.MurN--None,C17H30N2O12,454.17987,Amidase,0,1,G2,1,2,...,NaN,CC(=O)NC1C(OC2C(CO)OC(O)C(N)C2OC(C)C(=O)O)OC(C...,IGYLVGQPIFKPAO-UHFFFAOYSA-N,-4.7896,0,455.18715,477.16909,493.14303,228.09722,152.40057
DAG-MUR,(DAG)(MUR)-None,GlcNAcOAc.MurN--None,C19H32N2O13,496.19044,Amidase,0,2,G2,1,2,...,NaN,CC(=O)NC1C(OC2C(CO)OC(O)C(N)C2OC(C)C(=O)O)OC(C...,ZSDDRWVIQWSQPJ-UHFFFAOYSA-N,-4.2188,0,497.19772,519.17966,535.15360,249.10250,166.40409
NAG-NAM,(NAG)(NAM)-None,GlcNAc.MurNAc--None,C19H32N2O13,496.19044,Amidase,0,2,G2,1,2,...,NaN,CC(=O)NC1C(OC2C(CO)OC(O)C(NC(C)=O)C2OC(C)C(=O)...,WQKIVDUWFNRJHE-UHFFFAOYSA-N,-4.6122,0,497.19772,519.17966,535.15360,249.10250,166.40409
DAG-NAM,(DAG)(NAM)-None,GlcNAcOAc.MurNAc--None,C21H34N2O14,538.20100,Amidase,0,3,G2,1,2,...,NaN,CC(=O)NC1C(OC2C(CO)OC(O)C(NC(C)=O)C2OC(C)C(=O)...,SUTALUMKMRSKIF-UHFFFAOYSA-N,-4.0414,0,539.20828,561.19022,577.16416,270.10778,180.40761
NAG-DAM,(NAG)(DAM)-None,GlcNAc.MurNAcOAc--None,C21H34N2O14,538.20100,Amidase,0,3,G2,1,2,...,NaN,CC(=O)NC1C(OC2C(COC(C)=O)OC(O)C(NC(C)=O)C2OC(C...,GKOAKWSMCSQKMH-UHFFFAOYSA-N,-4.0414,0,539.20828,561.19022,577.16416,270.10778,180.40761
DAG-DAM,(DAG)(DAM)-None,GlcNAcOAc.MurNAcOAc--None,C23H36N2O15,580.21157,Amidase,0,4,G2,1,2,...,NaN,CC(=O)NC1C(OC2C(COC(C)=O)OC(O)C(NC(C)=O)C2OC(C...,UGHTWFWJTFCKTB-UHFFFAOYSA-N,-3.4706,0,581.21885,603.20079,619.17473,291.11307,194.41114
G-MUR,(G)(MUR)-None,GlcN.MurN--None,C15H28N2O11,412.16931,Amidase,0,0,G2,1,2,...,NaN,CC(OC1C(N)C(O)OC(CO)C1OC1OC(CO)C(O)C(O)C1N)C(=O)O,UBADYJPVCNDSPI-UHFFFAOYSA-N,-4.9670,0,413.17659,435.15853,451.13247,207.09194,138.39705


In [5]:
ligand_folder = data_folder / 'glycan_modifications' / 'pdb'
ligand_folder.mkdir(exist_ok=True)

for ligand_name, row in glycan_modifications.iterrows():
    raw_name = row['Name']
    smiles = row['SMILES']

    mol = Chem.MolFromSmiles(smiles)
    try:
        mol = generate_conformation(mol)
    except ValueError:
        print(f'Error for ligand: {raw_name}')
        raise

    output_path = ligand_folder / f'{ligand_name}.pdb'
    Chem.MolToPDBFile(mol, output_path.as_posix())

[17:29:09] Molecule does not have explicit Hs. Consider calling AddHs()
[17:29:09] Molecule does not have explicit Hs. Consider calling AddHs()
[17:29:09] Molecule does not have explicit Hs. Consider calling AddHs()
[17:29:09] Molecule does not have explicit Hs. Consider calling AddHs()
[17:29:09] Molecule does not have explicit Hs. Consider calling AddHs()
[17:29:09] Molecule does not have explicit Hs. Consider calling AddHs()
[17:29:09] Molecule does not have explicit Hs. Consider calling AddHs()
[17:29:09] Molecule does not have explicit Hs. Consider calling AddHs()
[17:29:09] Molecule does not have explicit Hs. Consider calling AddHs()
[17:29:09] Molecule does not have explicit Hs. Consider calling AddHs()
[17:29:09] Molecule does not have explicit Hs. Consider calling AddHs()
[17:29:09] Molecule does not have explicit Hs. Consider calling AddHs()
[17:29:09] Molecule does not have explicit Hs. Consider calling AddHs()
[17:29:09] Molecule does not have explicit Hs. Consider calling 

## Protenix results

In [5]:
protenix_lysozyme_folder = data_folder / 'glycan_modifications' / 'protenix__lysozyme_pulldown'

In [8]:
data = {
    'protein_domain': [],
    'ligand_name': [],
    'folder': [],
    'seed': [],
    'ptm': [],
    'iptm': [],
    'confidence': [],
}
domains_data = [
    (protenix_lysozyme_folder, 'Human_Lysozyme_C'), 
]
for result_folder, protein_domain in domains_data:
    for ligand_folder in result_folder.iterdir():
        if not ligand_folder.name.startswith(protein_domain):
            continue

        ligand_folder_name = ligand_folder.name
        try:
            ligand_name = ligand_folder_name.split('__')[1]
        except IndexError:
            print(ligand_folder_name)
            raise

        for seed_folder in ligand_folder.iterdir():
            seed = seed_folder.name
            for results_json_file in (seed_folder / 'predictions').glob('*_sample_0.json'):
                sample = results_json_file.name.split('_')[-1].replace('.json', '')
                with results_json_file.open() as f:
                    scores = json.load(f)
                
                ptm = scores['ptm']
                iptm = scores['iptm']
                confidence = 0.8 * iptm + 0.2 * ptm

                data['protein_domain'].append(protein_domain)
                data['ligand_name'].append(ligand_name)
                data['folder'].append(ligand_folder_name)
                data['seed'].append(seed)
                data['ptm'].append(ptm)
                data['iptm'].append(iptm)
                data['confidence'].append(confidence)

protenix_results_df = pd.DataFrame.from_dict(data).sort_values(
    'confidence', 
    ascending=False,
).drop_duplicates([
    'protein_domain', 
    'ligand_name'
]).set_index([
    'protein_domain',
    'ligand_name',
])
protenix_results_df.head(20)

folder      seed  \
protein_domain   ligand_name                                               
Human_Lysozyme_C G-MUR          Human_Lysozyme_C_P61626__G-MUR  seed_862   
                 NAG-MUR      Human_Lysozyme_C_P61626__NAG-MUR  seed_998   
                 G-DAM          Human_Lysozyme_C_P61626__G-DAM  seed_862   
                 DAG-MUR      Human_Lysozyme_C_P61626__DAG-MUR  seed_862   
                 NAG-DAM      Human_Lysozyme_C_P61626__NAG-DAM  seed_862   
                 DAG-DAM      Human_Lysozyme_C_P61626__DAG-DAM  seed_862   
                 G-NAM          Human_Lysozyme_C_P61626__G-NAM  seed_998   
                 DAG-NAM      Human_Lysozyme_C_P61626__DAG-NAM  seed_862   
                 NAG-NAM      Human_Lysozyme_C_P61626__NAG-NAM  seed_862   

                                   ptm      iptm  confidence  
protein_domain   ligand_name                                  
Human_Lysozyme_C G-MUR        0.957440  0.987072    0.981145  
                 NAG-MUR      0.957846  0.983700    0.978529  
                 G-DAM        0.965220  0.978546    0.975880  
                 DAG-MUR      0.955965  0.980187    0.975343  
                 NAG-DAM      0.957576  0.976485    0.972703  
                 DAG-DAM      0.959768  0.974485    0.971542  
                 G-NAM        0.963097  0.971376    0.969720  
                 DAG-NAM      0.954930  0.969175    0.966326  
                 NAG-NAM      0.960965  0.965639    0.964704

## AF3

In [ ]:
ligand_folder_af3 = data_folder / 'glycan_modifications' / 'af3__inputs'
ligand_folder_af3.mkdir(exist_ok=True)

for ligand_name, row in glycan_modifications.iterrows():
    raw_name = row['Name']
    smiles = row['SMILES']

    ccd_code = f'LIG-PG-{ligand_name}'.replace('_', '-')
    ligand_seq = {
        'ligand': {
            'id': 'B',
            'ccdCodes': [ccd_code],
        }
    }
    try:
        ccd_data = generate_ccd_from_smiles(smiles, ccd_code)
    except ValueError:
        print(f'Error for ligand: {raw_name}')
        raise

    data = {
        'name': ligand_name,
        'sequences': [ligand_seq],
        'userCCD': ccd_data,
        'dialect': 'alphafold3',
        'version': 1,
    }
    with (ligand_folder_af3 / f'{ligand_name}.json').open('w') as f_out:
        json.dump(
            data, 
            f_out,
            indent=True,
        )

[17:29:14] Molecule does not have explicit Hs. Consider calling AddHs()
[17:29:14] Molecule does not have explicit Hs. Consider calling AddHs()
[17:29:14] Molecule does not have explicit Hs. Consider calling AddHs()
[17:29:14] Molecule does not have explicit Hs. Consider calling AddHs()
[17:29:14] Molecule does not have explicit Hs. Consider calling AddHs()
[17:29:14] Molecule does not have explicit Hs. Consider calling AddHs()
[17:29:14] Molecule does not have explicit Hs. Consider calling AddHs()
[17:29:14] Molecule does not have explicit Hs. Consider calling AddHs()
[17:29:14] Molecule does not have explicit Hs. Consider calling AddHs()
[17:29:14] Molecule does not have explicit Hs. Consider calling AddHs()
[17:29:14] Molecule does not have explicit Hs. Consider calling AddHs()
[17:29:14] Molecule does not have explicit Hs. Consider calling AddHs()
[17:29:14] Molecule does not have explicit Hs. Consider calling AddHs()
[17:29:14] Molecule does not have explicit Hs. Consider calling 